In [ ]:
import torch

In [1]:
from GBM_MC_pricer import *

In [2]:
npath = 1000
strike = 99
s = 100
r = 0.06 #anual risk free rate
sigma = 0.2 #anual volatility
n_days = 10
#used for risk free rate over all the days when calculating option price for european option
T = n_days/365.0
T

0.0273972602739726

In [3]:
path, average_P, sterr_P = initiate_path(npath, s, r, sigma, n_days, strike, T)
# print(average_P, sterr_P)
stock_paths = path.iloc[:,0:-2].T

In [4]:
# path.iloc[:, :].T
stock_paths

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,98.561974,99.330279,101.127477,100.020024,100.752581,99.355318,101.394163,101.062719,99.979761,100.212598,...,99.315628,99.741347,101.532684,99.429409,101.494517,101.755011,100.196382,99.954738,98.844411,98.934970
2,98.072609,98.571407,102.669508,100.063679,99.723727,98.766726,103.024061,99.564819,100.355155,100.268344,...,98.447771,99.695266,102.611605,99.706836,100.984440,100.989218,100.082938,98.614423,98.860224,98.963649
3,97.320382,98.764219,103.631479,101.584841,100.003917,98.466192,102.861036,97.815838,98.595329,100.122471,...,100.108394,98.947229,104.067268,98.732518,102.097157,100.205669,99.886141,98.080086,101.219171,100.160168
4,97.169307,99.399468,104.438790,101.300288,99.819237,98.178103,102.540866,99.028964,97.554593,98.934502,...,100.444418,98.406201,101.655603,98.015080,101.080963,99.453480,100.566429,98.051933,101.250066,101.234346
5,97.456946,101.312097,102.955165,100.097494,98.803650,98.345100,102.248848,101.180753,99.118493,99.388901,...,99.665336,97.600142,103.573585,97.278064,99.450662,99.726604,100.338789,98.610487,101.792720,100.880396
6,98.041665,100.589310,103.262186,98.352291,99.163550,99.470257,101.153483,101.279278,97.412213,100.301389,...,98.721958,96.840353,104.512154,96.616590,100.815238,99.590198,100.284195,98.513547,100.823020,101.755496
7,98.625644,99.588899,103.731057,97.225292,99.022341,99.130391,102.007201,102.219628,96.370839,100.068603,...,99.284686,98.339509,105.147209,96.233166,101.879382,99.941574,101.769525,99.321947,100.094637,103.037520
8,100.426378,98.496874,103.913377,97.919574,100.049113,98.666637,101.254274,101.477247,96.790670,101.084512,...,98.587530,97.379740,106.378493,97.562532,102.874220,99.533701,103.537464,99.215012,99.664829,103.649986
9,99.664618,97.801802,105.970393,98.064031,99.816904,97.480516,100.624261,99.751733,96.666058,101.003525,...,101.550828,97.280537,106.259014,95.506608,102.572484,98.099031,104.837232,98.696913,99.421624,103.919599


In [10]:
def generate_covariance_from_correlation(cor_mat, vol_list, dt):
    vol_diag_mat = np.diag(vol_list)
    cov_mat = np.dot(np.dot(vol_diag_mat,cor_mat),vol_diag_mat)*dt
    return cov_mat


def multi_variate_gbm_simulation(no_of_paths, no_of_exercise_days, no_of_assets, 
                                 curr_stock_price, r, vol_list, cov_mat, dt):
    zero_mean = np.zeros(no_of_assets)
    
    #Antithetic variance reduction for monte-carlo
    w_matx = np.random.multivariate_normal(zero_mean, cov_mat, (int(no_of_paths/2),no_of_exercise_days))
    w_mat= np.concatenate((w_matx,- w_matx),axis=0)
    w_mat = w_mat.reshape(no_of_paths, no_of_exercise_days)
    
    sim_ln_stock_mat = np.zeros((no_of_paths, no_of_exercise_days + 1))
    sim_ln_stock_mat[:,0] = np.log(curr_stock_price)
  
    base_drift =  (r - 0.5 * np.square(vol_list[0])) * dt
    
    for day in range(1, no_of_exercise_days+1):
        curr_drift = sim_ln_stock_mat[:,day-1] + base_drift
        sim_ln_stock_mat[:,day] = curr_drift + w_mat[:,day-1]

    sim_stock_mat = np.exp(sim_ln_stock_mat)
    
    return sim_stock_mat

In [11]:
no_of_assets = 1
cor_mat = [[1]]
vol_list = np.array([0.3])
curr_stock_price = 1*np.ones(no_of_assets)
t = 0.2
k = 1
barrier_list = [0.85, 0.87, 0.89, 0.91, 0.93, 0.95, 0.97, 0.99]
# barrier_list = [0.85]
no_of_barrier_points = 5
finer_path_multiplier = 1
r = 0.1
w = np.array([1])
w = w.reshape(-1,1)
no_of_paths =50000
Notional = 100

#Generate covariance matrix 
dt = t/no_of_barrier_points
cov_mat = generate_covariance_from_correlation(cor_mat, vol_list, dt)



#Generating stock prices for training neural network
sim_stock_mat = multi_variate_gbm_simulation(no_of_paths, no_of_barrier_points, no_of_assets, 
                                                     curr_stock_price, r, vol_list, cov_mat, dt)

In [15]:
len(sim_stock_mat[:, 4])

50000